# Dependencies

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
%matplotlib inline

# Preprocessing the Data

In [2]:
# read csv into dataframe
filepath = "Austin Properties 2019.1.6.csv"
csv_df = pd.read_csv(filepath)

In [3]:
# check the column names
csv_df.columns

Index(['Address', 'Building Name', 'Zip Code', 'Bed', 'Bath', 'Avg SF',
       '# Units', 'Mix %', 'Units Available Units', 'Units Available Percent',
       'Avg Asking Rent/Unit', 'Avg Asking Rent/SF', 'Avg Effective Rent/Unit',
       'Avg Effective Rent/SF', 'Concessions %', 'Year Built',
       'Year Renovated', 'Units', 'Lande (Acres)', 'Vacancy %', 'Washer/Dryer',
       'Walk in Closet', 'Hardwood/Vinyl Floor', 'Walk Score', 'Transit Score',
       '1 Mile Population', '1 Mile Median Household Income',
       'Miles from Domain', 'Miles from Downtown', 'Unnamed: 29'],
      dtype='object')

In [4]:
csv_df = pd.get_dummies(data=csv_df, columns=['Washer/Dryer'])

In [5]:
# select certain variables
original_df = csv_df[['Bed', 'Bath', 'Avg SF', 'Concessions %', 'Year Built', 
   'Walk in Closet', 'Hardwood/Vinyl Floor',
   'Washer/Dryer_Yes but not in unit', 'Washer/Dryer_Yes in unit', 'Washer/Dryer_no',
   'Walk Score', 'Transit Score', '1 Mile Population', '1 Mile Median Household Income',
   'Miles from Domain', 'Miles from Downtown', 
   'Avg Effective Rent/Unit']]

In [6]:
# check na values
original_df.count()

Bed                                  5543
Bath                                 5544
Avg SF                               5544
Concessions %                        5544
Year Built                           5544
Walk in Closet                       5544
Hardwood/Vinyl Floor                 5544
Washer/Dryer_Yes but not in unit    11079
Washer/Dryer_Yes in unit            11079
Washer/Dryer_no                     11079
Walk Score                           5526
Transit Score                        5526
1 Mile Population                    5490
1 Mile Median Household Income       5490
Miles from Domain                    5544
Miles from Downtown                  5544
Avg Effective Rent/Unit              5541
dtype: int64

In [7]:
# process na values
austin_properties_df = original_df.dropna()
# check the result of na value processing
austin_properties_df.count()

Bed                                 5468
Bath                                5468
Avg SF                              5468
Concessions %                       5468
Year Built                          5468
Walk in Closet                      5468
Hardwood/Vinyl Floor                5468
Washer/Dryer_Yes but not in unit    5468
Washer/Dryer_Yes in unit            5468
Washer/Dryer_no                     5468
Walk Score                          5468
Transit Score                       5468
1 Mile Population                   5468
1 Mile Median Household Income      5468
Miles from Domain                   5468
Miles from Downtown                 5468
Avg Effective Rent/Unit             5468
dtype: int64

In [8]:
# check the data type
austin_properties_df.dtypes

Bed                                 float64
Bath                                float64
Avg SF                              float64
Concessions %                       float64
Year Built                          float64
Walk in Closet                      float64
Hardwood/Vinyl Floor                float64
Washer/Dryer_Yes but not in unit      uint8
Washer/Dryer_Yes in unit              uint8
Washer/Dryer_no                       uint8
Walk Score                          float64
Transit Score                       float64
1 Mile Population                   float64
1 Mile Median Household Income      float64
Miles from Domain                   float64
Miles from Downtown                 float64
Avg Effective Rent/Unit             float64
dtype: object

In [9]:
# overview of the new dataframe
austin_properties_df.head()

,Bed,Bath,Avg SF,Concessions %,Year Built,Walk in Closet,Hardwood/Vinyl Floor,Washer/Dryer_Yes but not in unit,Washer/Dryer_Yes in unit,Washer/Dryer_no,Walk Score,Transit Score,1 Mile Population,1 Mile Median Household Income,Miles from Domain,Miles from Downtown,Avg Effective Rent/Unit
0,1.0,1.0,560.0,0.01,2016.0,1.0,1.0,0,1,0,88.0,44.0,20115.0,70385.0,12.6,2.1,1528.0
1,1.0,1.0,612.0,0.01,2016.0,1.0,1.0,0,1,0,88.0,44.0,20115.0,70385.0,12.6,2.1,1633.0
2,1.0,1.0,629.0,0.01,2016.0,1.0,1.0,0,1,0,88.0,44.0,20115.0,70385.0,12.6,2.1,1740.0
3,1.0,1.0,774.0,0.01,2016.0,1.0,1.0,0,1,0,88.0,44.0,20115.0,70385.0,12.6,2.1,1778.0
4,1.0,1.0,778.0,0.01,2016.0,1.0,1.0,0,1,0,88.0,44.0,20115.0,70385.0,12.6,2.1,1852.0


# Determine X and y for Machine Learning

In [10]:
X = austin_properties_df.iloc[:, :-1]
y = austin_properties_df['Avg Effective Rent/Unit']

# Splitting the data into training and testing sets

In [11]:
from sklearn.model_selection import train_test_split

# 80/20 training and testing data split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=5)

In [12]:
# check the shape of training and test data
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(4374, 16)
(1094, 16)
(4374,)
(1094,)


# Model (I) - Linear Regression

In [13]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

In [14]:
# train the model
lin_model = LinearRegression()
lin_model.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [15]:
# model evaluation
y_test_predicted = lin_model.predict(X_test)
mse_test = mean_squared_error(y_test, y_test_predicted)
r2_test = r2_score(y_test, y_test_predicted)
print(f"Mean Squared Error (MSE): {mse_test}")
print(f"R-quared (R2): {r2_test}")

Mean Squared Error (MSE): 296512.9266368084
R-quared (R2): 0.5344241825987113


# Model (II) - Polynomial Regression

In [16]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline

In [28]:
poly_model = Pipeline([
    ('a', PolynomialFeatures(degree=2)),
    ('b', LinearRegression(fit_intercept=False))
])

In [18]:
poly_model.fit(X_train, y_train)
poly_model.score(X_test, y_test)

0.44883007398009794

# Random Forest

In [19]:
from sklearn.ensemble import RandomForestRegressor

In [20]:
rf = RandomForestRegressor(max_depth=2, random_state=0, n_estimators=200)

In [21]:
rf.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=2,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=200, n_jobs=None,
           oob_score=False, random_state=0, verbose=0, warm_start=False)

In [22]:
print(rf.feature_importances_)

[0.         0.         0.71200253 0.00500308 0.00223621 0.01537523
 0.00364181 0.         0.         0.         0.00489611 0.21074496
 0.01393217 0.01617954 0.01598836 0.        ]


# Model (III) - Neural Network Model

In [23]:
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

Using TensorFlow backend.


In [24]:
print(len(X_train.columns))
print(X_train.columns)

16
Index(['Bed', 'Bath', 'Avg SF', 'Concessions %', 'Year Built',
       'Walk in Closet', 'Hardwood/Vinyl Floor',
       'Washer/Dryer_Yes but not in unit', 'Washer/Dryer_Yes in unit',
       'Washer/Dryer_no', 'Walk Score', 'Transit Score', '1 Mile Population',
       '1 Mile Median Household Income', 'Miles from Domain',
       'Miles from Downtown'],
      dtype='object')


In [25]:
# define base model
def baseline_model():
    # create model
    model = Sequential()
    model.add(Dense(13, input_dim=16, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, kernel_initializer='normal'))
    
    # compile model
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

In [26]:
# fix random seed for reproducibility
seed = 7
np.random.seed(seed)
# evaluate model with standardized dataset
estimator = KerasRegressor(build_fn=baseline_model, epochs=100, batch_size=5, verbose=0)

In [27]:
kfold = KFold(n_splits=10, random_state=seed)
results = cross_val_score(estimator, X_train, y_train, cv=kfold)
print(f"Results: {results.mean()}, {results.std()} MSE")

Results: -297837.78843523527, 109574.00568921285 MSE


# Residuals

In [ ]:
plt.scatter(y_test_predicted, y_test_predicted-y_test)
plt.hlines(y=0, xmin=y_test_predicted.min(), xmax=y_test_predicted.max())
plt.show()

In [ ]:
model.predict()